## "The mother ship" 
This notebook organizes inpts and function calls for creating the monte carlo csv file

##### Import functions neccessary for calling scripts

In [1]:
# import monte-carlo functions
from get_date import get_date
from get_lat_lon_indices import get_lat_lon_indices
from get_vessel_type import get_vessel_type
from get_vte_probability import get_vte_probability
from get_oil_capacity import get_oil_capacity

# import python functions needed for input variables
from datetime import datetime, date, timedelta
import arrow

##### These are the input variables needed to run the functions for the monte-carlo setup

In [2]:
# spill date
start_date = datetime(2015, 1, 1, 0, 30)
end_date   = datetime(2018, 12, 31, 23, 55)
delta_time = timedelta(hours = 1)

# spill location
ais_data_year = 2018
n_locations = 1  #desired number of latitude and longitude values
geotiff_directory = '/Users/rmueller/Data/MIDOSS/AIS/ShipTrackDensityGeoTIFFs/'
upsample_factor = 1 # GeoTIFF lat/lon will be sampled at 1/upsample_factor [km] resolution
vessel_types      = ["tanker", "atb", "barge", "cargo", "cruise",
                     "ferry", "fishing", "smallpass", "other"]

##### Imports and definitions for testing scripts and validating output

In [3]:
# these aren't used or needed for function calls
from rasterio.enums import Resampling
import rasterio as rio

#### Get the spill date

In [4]:
# get vessel traffic exposure probability by month
vte_probability = get_vte_probability( geotiff_directory )

spill_date = get_date( start_date, 
             end_date, 
             vte_probability,
             delta_time
            )
spill_month = spill_date.month
spill_month_name  = arrow.get(spill_date).format('MMMM') # string
f'{spill_month:02.0f}'

'06'

####  Get spill lat/lon value(s) as well as index values for extracting vessel time exposure values from GeoTIFFS

In [5]:
[lats, lons, x_index, y_index, data_val] = get_lat_lon_indices(geotiff_directory, spill_month, n_locations, upsample_factor)    

Randomly selecting spill location from all-traffic GeoTIFF:
...Creating 2D array of lat/lon strings (this may take a moment)
...Selecting 1 location(s)
...Creating vectors of lat/lon pairs and x-index/y-index pairs
That's a wrap, folks!


In [6]:
lats

array([-124.9243])

In [7]:
lons

array([ 50.18442])

In [8]:
x_index

array([149])

In [9]:
y_index

array([373])

In [10]:
data_val

array([ 3.11596155])

In [11]:
# load geotiff and access "data_val" with [x_index, y_index] to verify that the output is equivalent

traffic_reader = rio.open(f'{geotiff_directory}all_2018_{spill_month:02.0f}.tif')

# dataset closes automatically using the method below

with traffic_reader as dataset:
    # resample data to target shape
    data = dataset.read(1,
        out_shape=(
            dataset.count,
            int(dataset.height * upsample_factor),
            int(dataset.width * upsample_factor)
        ),
        resampling=Resampling.bilinear
    )

    # scale image transform
    transform = dataset.transform * dataset.transform.scale(
        (dataset.width / data.shape[-1]),
        (dataset.height / data.shape[-2])
    )

print(f'compare output data_val = {data_val} hours/km^2 with {data[x_index,y_index]} hours/km^2')    

compare output data_val = [ 3.11596155] hours/km^2 with [ 3.11596155] hours/km^2


####  Get vessel type 

In [12]:
vessel_type = get_vessel_type(geotiff_directory, 
                              vessel_types, 
                              ais_data_year, 
                              n_locations, 
                              spill_month,  # from get_date
                              x_index,      # from get_lat_lon_indices
                              y_index       # from get_lat_lon_indices
                              )
vessel_type

[array(['other'],
       dtype='<U9')]

In [4]:
# this is just a test of the get_oil_capacity script
for vessel in vessel_types:
    [fuel, tank] = get_oil_capacity(60, vessel)
    print(f'{vessel} [fuel, tank]: {fuel:4.2e} m^3, {tank:4.2e} m^3')



tanker [fuel, tank]: 7.35e+05 m^3, 1.81e+07 m^3
atb [fuel, tank]: 4.50e+06 m^3, 3.84e+06 m^3
barge [fuel, tank]: 4.50e+06 m^3, 3.84e+06 m^3
cargo [fuel, tank]: 7.35e+05 m^3, 0.00e+00 m^3
cruise [fuel, tank]: 4.01e+05 m^3, 0.00e+00 m^3
ferry [fuel, tank]: 4.84e+04 m^3, 0.00e+00 m^3
fishing [fuel, tank]: 3.23e+18 m^3, 0.00e+00 m^3
smallpass [fuel, tank]: 2.43e+03 m^3, 0.00e+00 m^3
other [fuel, tank]: 2.43e+03 m^3, 0.00e+00 m^3
